In [1]:
from scipy.stats import mode
import time
import statistics
import operator
import cv2
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def get_data(read_path,filter_count=0,filter_type='m'):
    
    #Load the image in black and white (0 - b/w, 1 - color).
    img = cv2.imread(read_path, 0)

    #Get the height and width of the image.
    h, w = img.shape[:2]

    rects=rects1=rects2=[]
    #Invert the image to be white on black for compatibility with findContours function.
    imgray = 255 - img
    #ret, imgray = cv2.threshold(imgray, 80, 255, cv2.THRESH_BINARY_INV)
    size=9+2*filter_count
    print("1")
    if (filter_type=='m'):
        thresh1 = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,size,2)

        #Find all the contours in thresh. 
        contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        #Calculate bounding rectangles for each contour.
        rects1 = [cv2.boundingRect(cnt) for cnt in contours]
        
        imgray2 = cv2.blur(imgray,(5,5))
        thresh2 = cv2.adaptiveThreshold(imgray2, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,size,2)
        #Find all the contours in thresh. 
        contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        rects2 = [cv2.boundingRect(cnt) for cnt in contours]
        
        
    else:
        thresh1 = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,size,2)
        
        #Find all the contours in thresh.
        contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        #Calculate bounding rectangles for each contour.
        rects1 = [cv2.boundingRect(cnt) for cnt in contours]
        
        imgray2 = cv2.blur(imgray,(5,5))
        thresh2 = cv2.adaptiveThreshold(imgray2, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,size,2)
        #Find all the contours in thresh.
        contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        rects2 = [cv2.boundingRect(cnt) for cnt in contours]

    print(len(rects1),len(rects2))
    
# del all tuple of list who has less than or greater than 50,80 size


    for i in rects1:
        print(i)
        if (i[2]>50 and i[2]<80 and i[3]>50 and i[3]<80 and i not in rects):
            rects.append(i)

    for i in rects2:
        if (i[2]>50 and i[2]<80 and i[3]>50 and i[3]<80 and i not in rects):
            rects.append(i)
    
    rects_final=rects.copy()
    for i in range(0,len(rects)-1):
        try:
            for j in range(i+1,len(rects)):
                if (abs(rects[i][0] - rects[j][0]) < 10 and abs(rects[i][1] - rects[j][1]) < 10):
                    if (rects[i][2]  < rects[j][2] and rects[i][3] < rects[j][3]):
                        rects_final.remove(rects[j])
                    else:
                        rects_final.remove(rects[i])
        except:
            continue

    
    rects=rects_final.copy()


        
    #Calculate the combined bounding rectangle points.
    top_x = min([x for (x, y, w, h) in rects])
    top_y = min([y for (x, y, w, h) in rects])
    bottom_x = max([x+w for (x, y, w, h) in rects])
    bottom_y = max([y+h for (x, y, w, h) in rects])
#     print(top_x)
#     print(top_y)
#     print(bottom_x)
#     print(bottom_y)
    
    #h_mode = statistics.mode([h for (x, y, w, h) in rects2])
    h_mode = mode([h for (x, y, w, h) in rects2])[0]
    
    #print(h_mode)
    
    n=int(((bottom_y-top_y)/int(h_mode)))
    #gap=(bottom_y-top_y)-(h_mode*20)
    #n=int(((bottom_y-top_y)-(gap))/h_mode)
    #print('Mode',h_mode)
    #print(n)
    
    temp=[]
    b = [[] for i in range(0, n+1)]
    for j in range (1,n+1):
        for i in rects2:
            x=i[0]
            y=i[1]
            w=i[2]
            h=i[3]
            if ((y > bottom_y-h_mode*j-h_mode/2) and (i not in temp)):
                b[j].append(i)
                temp.append(i)
            b[j]=sorted(b[j],key=lambda x:(-x[0]))
    return(imgray,b)

In [3]:
df=pd.read_excel(r"C:\Users\Rahul\Desktop\Capstone\OCR\scan copy\Labels.xlsx",index_col="Sr. No.")

In [6]:
read_path=r"C:\Users\Rahul\Desktop\Capstone\OCR\scan copy\scan_19_1"+"\\"
class_path=r'C:\Users\Rahul\Desktop\Capstone\OCR\data\results_19_1'+'\\'
imgsave=1
for folder in range(1,621):
    class_folder=class_path+str(df.loc[folder,"Labels2"])
    if not os.path.exists(class_folder):
        os.makedirs(class_folder) 
m=g=0
c=[]

for i in range(2,30):
    if (i==18 or i==21):
        continue
    if (m==10 and g==10 and len(c)!=620):
        print("Skipped image is ", (i-1))
    m=g=0
    c=[]
    while((len(c)!=620) and (m!=10 or g!=10)):
        c=[]
        if (m<10 and g==0):
            m+=1
            imgray,b=get_data(read_path+str(i)+".jpg",filter_count=m,filter_type='m')
            #print(m)
        elif (m==10 and g<10):
            g+=1
            imgray,b=get_data(read_path+str(i)+".jpg",filter_count=g,filter_type='g')
            #print(g)  
        for row in b:
            for box in row:
                try:
                    w=box[2]
                    h=box[3]
                    #print(w)
                    if ((w>60 and w<80) and (h>60 and h <80)):
                        c.append(box)
                        #print(c)
                except:
                    continue
                    
        print(len(c))
        if (len(c)==620):
            print("here")
            count=1
            for box in c:
                x=box[0]
                y=box[1]
                w=box[2]
                h=box[3]
                class_folder=class_path+str(df.loc[count,"Labels2"])
                cv2.imwrite(class_folder+'\\'+str(i)+" img "+str(imgsave)+".jpg",imgray[y:y+h,x:x+w])
                imgsave+=1
                time.sleep(0.005)
                count+=1

1
22732 6826
(1395, 3471, 33, 4)
(1333, 3471, 59, 5)
(1297, 3471, 12, 3)
(1236, 3471, 3, 3)
(1217, 3471, 9, 3)
(1204, 3471, 3, 3)
(1176, 3471, 10, 4)
(1038, 3471, 9, 3)
(1294, 3470, 3, 3)
(1207, 3470, 6, 3)
(151, 3470, 6, 3)
(1463, 3468, 3, 3)
(1414, 3468, 3, 3)
(1383, 3468, 3, 3)
(1332, 3468, 5, 3)
(1343, 3467, 4, 3)
(1375, 3466, 4, 3)
(1287, 3466, 3, 4)
(1207, 3466, 7, 4)
(1407, 3465, 5, 3)
(1241, 3465, 6, 5)
(1162, 3465, 7, 3)
(1031, 3465, 4, 3)
(159, 3465, 6, 5)
(95, 3465, 7, 3)
(89, 3465, 4, 3)
(76, 3465, 3, 3)
(1444, 3464, 13, 5)
(1439, 3464, 5, 5)
(1399, 3464, 8, 5)
(1386, 3464, 7, 4)
(1347, 3464, 6, 5)
(1210, 3464, 8, 4)
(1170, 3464, 4, 3)
(1037, 3464, 4, 5)
(1007, 3464, 12, 4)
(999, 3464, 5, 5)
(136, 3464, 9, 4)
(111, 3464, 18, 5)
(47, 3464, 4, 4)
(31, 3464, 8, 4)
(1413, 3463, 5, 4)
(1383, 3463, 4, 3)
(1371, 3463, 5, 3)
(1365, 3463, 10, 7)
(1357, 3463, 6, 5)
(1352, 3463, 6, 5)
(1343, 3463, 5, 3)
(1304, 3463, 25, 6)
(1265, 3463, 3, 3)
(1225, 3463, 6, 4)
(1207, 3463, 4, 3)
(1188

(122, 3278, 7, 3)
(111, 3278, 3, 3)
(103, 3278, 7, 3)
(79, 3278, 5, 3)
(71, 3278, 7, 5)
(52, 3278, 10, 6)
(1319, 3277, 10, 5)
(127, 3277, 10, 3)
(78, 3277, 3, 3)
(66, 3276, 6, 5)
(1215, 3275, 5, 5)
(146, 3275, 7, 3)
(74, 3275, 4, 4)
(1319, 3274, 5, 4)
(92, 3274, 5, 4)
(1301, 3273, 8, 5)
(1175, 3273, 11, 4)
(119, 3273, 18, 4)
(100, 3273, 9, 4)
(63, 3273, 4, 4)
(1345, 3272, 8, 3)
(157, 3272, 4, 4)
(135, 3272, 11, 3)
(2452, 3271, 5, 3)
(2439, 3271, 12, 5)
(1367, 3271, 3, 3)
(1319, 3271, 10, 4)
(1300, 3271, 5, 3)
(1194, 3271, 9, 4)
(145, 3271, 6, 4)
(127, 3271, 10, 3)
(92, 3271, 5, 3)
(41, 3271, 8, 5)
(1327, 3270, 3, 3)
(1318, 3270, 3, 3)
(1311, 3270, 3, 3)
(119, 3270, 5, 4)
(2450, 3269, 4, 4)
(111, 3269, 8, 6)
(47, 3269, 14, 7)
(34, 3269, 8, 7)
(95, 3268, 4, 3)
(1318, 3267, 3, 3)
(1303, 3267, 7, 5)
(111, 3267, 3, 3)
(63, 3267, 32, 10)
(1340, 3266, 5, 4)
(1311, 3266, 3, 4)
(1296, 3266, 3, 4)
(132, 3266, 3, 3)
(29, 3266, 6, 7)
(1351, 3265, 3, 3)
(1295, 3265, 3, 3)
(1208, 3265, 4, 3)
(157, 3

(2421, 2857, 4, 4)
(111, 2857, 4, 4)
(2452, 2856, 5, 3)
(844, 2856, 4, 3)
(841, 2856, 4, 4)
(93, 2856, 4, 4)
(63, 2856, 5, 4)
(2419, 2855, 4, 3)
(113, 2855, 8, 4)
(103, 2855, 5, 4)
(73, 2855, 6, 5)
(55, 2855, 5, 4)
(48, 2855, 5, 4)
(2439, 2854, 15, 4)
(97, 2854, 4, 3)
(92, 2854, 5, 3)
(108, 2853, 5, 4)
(76, 2853, 4, 4)
(2431, 2852, 9, 6)
(2423, 2852, 7, 5)
(127, 2852, 5, 3)
(2439, 2851, 19, 4)
(111, 2851, 18, 4)
(100, 2851, 5, 6)
(47, 2851, 10, 5)
(69, 2850, 4, 4)
(56, 2850, 11, 5)
(94, 2849, 3, 3)
(79, 2849, 13, 8)
(2445, 2848, 4, 3)
(85, 2847, 4, 3)
(43, 2847, 5, 10)
(95, 2846, 6, 5)
(46, 2845, 3, 3)
(2436, 2844, 7, 4)
(87, 2844, 6, 3)
(43, 2844, 5, 3)
(2451, 2843, 5, 4)
(110, 2843, 3, 3)
(80, 2843, 6, 4)
(66, 2843, 4, 3)
(51, 2843, 6, 4)
(132, 2842, 6, 6)
(118, 2842, 13, 7)
(102, 2842, 3, 4)
(73, 2841, 6, 5)
(2438, 2840, 5, 4)
(2424, 2840, 6, 5)
(85, 2840, 12, 5)
(2444, 2839, 5, 5)
(105, 2839, 12, 10)
(95, 2839, 3, 3)
(63, 2839, 7, 5)
(2426, 2838, 5, 3)
(119, 2838, 3, 3)
(31, 2838, 

(629, 2498, 4, 4)
(540, 2498, 5, 4)
(481, 2498, 3, 4)
(647, 2497, 3, 3)
(552, 2497, 6, 4)
(35, 2497, 3, 3)
(783, 2496, 11, 4)
(759, 2496, 10, 3)
(725, 2496, 4, 4)
(677, 2496, 4, 3)
(623, 2496, 5, 3)
(615, 2496, 4, 5)
(485, 2496, 4, 4)
(47, 2496, 4, 4)
(36, 2496, 13, 6)
(804, 2495, 11, 6)
(798, 2495, 5, 5)
(782, 2495, 3, 3)
(767, 2495, 4, 3)
(727, 2495, 4, 3)
(719, 2495, 7, 4)
(687, 2495, 3, 3)
(681, 2495, 3, 3)
(611, 2495, 4, 3)
(599, 2495, 5, 4)
(592, 2495, 8, 4)
(586, 2495, 3, 3)
(568, 2495, 4, 4)
(547, 2495, 6, 4)
(527, 2495, 5, 3)
(487, 2495, 4, 4)
(832, 2494, 6, 3)
(820, 2494, 4, 3)
(815, 2494, 4, 3)
(769, 2494, 11, 8)
(738, 2494, 3, 3)
(732, 2494, 9, 6)
(729, 2494, 3, 3)
(632, 2494, 25, 8)
(574, 2494, 3, 3)
(566, 2494, 3, 3)
(47, 2494, 4, 3)
(871, 2493, 3, 3)
(763, 2493, 4, 4)
(759, 2493, 3, 4)
(730, 2493, 3, 3)
(615, 2493, 18, 4)
(456, 2493, 7, 3)
(697, 2493, 5, 3)
(524, 2492, 5, 4)
(33, 2491, 3, 3)
(47, 2490, 5, 4)
(35, 2490, 6, 4)
(34, 2490, 3, 3)
(47, 2487, 3, 3)
(36, 2487, 6

(771, 2271, 5, 3)
(725, 2271, 4, 3)
(711, 2271, 8, 8)
(708, 2271, 5, 3)
(652, 2271, 4, 4)
(623, 2271, 4, 4)
(617, 2271, 3, 3)
(587, 2271, 6, 3)
(557, 2271, 7, 6)
(548, 2271, 7, 4)
(530, 2271, 9, 6)
(495, 2271, 6, 4)
(487, 2271, 6, 3)
(252, 2271, 2048, 104)
(839, 2270, 5, 3)
(821, 2270, 18, 9)
(684, 2270, 5, 3)
(672, 2270, 20, 6)
(629, 2270, 16, 7)
(860, 2269, 12, 6)
(818, 2269, 3, 3)
(731, 2269, 33, 9)
(735, 2269, 10, 5)
(658, 2269, 10, 6)
(655, 2269, 4, 4)
(646, 2269, 4, 5)
(595, 2269, 18, 7)
(570, 2269, 7, 4)
(559, 2269, 10, 4)
(532, 2269, 5, 3)
(529, 2269, 3, 3)
(835, 2268, 3, 3)
(819, 2268, 3, 3)
(791, 2268, 22, 4)
(709, 2268, 17, 6)
(697, 2268, 13, 6)
(486, 2268, 11, 5)
(856, 2267, 6, 6)
(764, 2267, 13, 7)
(645, 2267, 4, 4)
(567, 2267, 5, 4)
(550, 2267, 4, 4)
(381, 2267, 4, 4)
(863, 2266, 5, 4)
(739, 2266, 16, 5)
(717, 2266, 4, 3)
(705, 2266, 5, 4)
(649, 2266, 8, 5)
(583, 2266, 3, 4)
(575, 2266, 7, 7)
(837, 2265, 5, 4)
(799, 2265, 6, 4)
(790, 2265, 3, 5)
(777, 2265, 7, 7)
(755, 22

...

(2004, 815, 3, 3)
(1417, 816, 5, 3)
(1174, 816, 1, 1)
(1157, 816, 1, 1)
(841, 815, 3, 3)
(831, 816, 1, 1)
(801, 815, 5, 6)
(773, 815, 4, 4)
(672, 815, 4, 5)
(666, 815, 3, 3)
(631, 815, 3, 3)
(274, 816, 20, 19)
(43, 815, 7, 4)
(42, 815, 3, 3)
(2453, 814, 4, 3)
(1993, 814, 21, 22)
(1601, 815, 6, 5)
(1594, 815, 3, 5)
(1030, 815, 1, 1)
(836, 814, 5, 3)
(828, 815, 1, 1)
(758, 815, 2, 1)
(607, 814, 4, 3)
(535, 814, 3, 3)
(276, 815, 1, 1)
(95, 814, 7, 3)
(76, 814, 8, 3)
(63, 814, 13, 5)
(53, 814, 4, 5)
(2195, 814, 1, 1)
(1597, 814, 3, 5)
(1149, 814, 1, 1)
(839, 813, 4, 4)
(830, 814, 1, 1)
(766, 814, 3, 2)
(2458, 812, 3, 4)
(1147, 813, 1, 1)
(583, 813, 1, 2)
(284, 812, 4, 3)
(274, 813, 1, 1)
(2441, 811, 12, 5)
(1990, 812, 26, 26)
(1175, 812, 1, 2)
(966, 812, 2, 2)
(389, 812, 1, 1)
(276, 812, 1, 1)
(84, 811, 11, 6)
(47, 811, 4, 4)
(668, 810, 8, 7)
(439, 811, 1, 1)
(95, 810, 6, 4)
(71, 810, 4, 4)
(56, 810, 3, 3)
(2048, 810, 19, 24)
(1144, 810, 2, 1)
(649, 810, 1, 2)
(632, 810, 3, 2)
(583, 810, 1

(232, 536, 3, 3)
(53, 536, 18, 8)
(2446, 535, 3, 3)
(2431, 535, 11, 3)
(2422, 535, 3, 3)
(797, 536, 52, 49)
(769, 536, 4, 2)
(731, 536, 51, 49)
(665, 536, 52, 48)
(599, 536, 52, 49)
(37, 535, 14, 9)
(2450, 534, 5, 6)
(700, 535, 1, 1)
(619, 535, 1, 1)
(534, 535, 51, 49)
(544, 535, 1, 1)
(535, 535, 6, 1)
(469, 535, 52, 49)
(405, 535, 50, 49)
(340, 535, 50, 49)
(275, 535, 50, 49)
(67, 534, 6, 5)
(28, 535, 5, 3)
(2234, 533, 64, 62)
(2169, 533, 64, 62)
(2103, 533, 64, 61)
(2038, 533, 64, 61)
(1973, 532, 64, 62)
(1907, 532, 64, 62)
(1840, 532, 65, 62)
(1775, 532, 64, 61)
(1709, 532, 64, 61)
(1643, 532, 64, 61)
(2455, 531, 3, 5)
(1577, 531, 65, 62)
(1512, 531, 64, 62)
(1446, 531, 64, 62)
(1381, 531, 64, 61)
(1316, 531, 64, 61)
(1250, 531, 64, 61)
(1185, 531, 64, 61)
(71, 531, 7, 5)
(56, 531, 13, 6)
(44, 531, 13, 9)
(35, 531, 6, 4)
(2447, 530, 3, 5)
(2428, 530, 15, 7)
(1119, 530, 64, 62)
(1054, 530, 64, 62)
(988, 530, 64, 62)
(922, 530, 65, 61)
(856, 530, 65, 61)
(790, 530, 65, 61)
(725, 530, 

(2375, 114, 10, 3)
(2369, 114, 3, 3)
(2339, 114, 5, 5)
(2320, 114, 6, 4)
(1327, 114, 4, 3)
(1319, 114, 3, 3)
(1303, 114, 4, 3)
(1217, 114, 4, 5)
(1187, 114, 5, 3)
(1175, 114, 15, 8)
(1143, 114, 21, 9)
(1151, 114, 3, 3)
(1105, 114, 6, 4)
(1095, 114, 6, 5)
(1029, 114, 6, 5)
(1024, 114, 5, 5)
(1010, 114, 15, 6)
(991, 114, 6, 4)
(2438, 113, 6, 5)
(2407, 113, 9, 4)
(1423, 113, 3, 3)
(1403, 113, 5, 5)
(1378, 113, 5, 4)
(1370, 113, 3, 3)
(1337, 113, 4, 4)
(1270, 113, 3, 4)
(1263, 113, 3, 3)
(1233, 113, 14, 8)
(1195, 113, 7, 4)
(1135, 113, 5, 4)
(1125, 113, 4, 3)
(1085, 113, 4, 3)
(1054, 113, 3, 6)
(1049, 113, 4, 6)
(966, 113, 3, 6)
(653, 113, 4, 4)
(612, 113, 4, 4)
(608, 113, 4, 4)
(2346, 112, 7, 4)
(2340, 112, 3, 3)
(2325, 112, 4, 4)
(2317, 112, 5, 4)
(1447, 112, 3, 3)
(1359, 112, 10, 3)
(1342, 112, 4, 4)
(1282, 112, 3, 3)
(1247, 112, 4, 3)
(1205, 112, 4, 4)
(1115, 112, 5, 5)
(1071, 112, 3, 4)
(989, 112, 4, 3)
(983, 112, 4, 3)
(754, 112, 4, 4)
(675, 112, 11, 8)
(661, 112, 4, 5)
(615, 112, 10

(793, 71, 3, 3)
(782, 71, 3, 3)
(778, 71, 4, 3)
(728, 71, 6, 4)
(714, 71, 5, 5)
(705, 71, 3, 3)
(659, 71, 18, 10)
(598, 71, 14, 9)
(585, 71, 11, 5)
(540, 71, 5, 3)
(519, 71, 3, 3)
(515, 71, 5, 3)
(511, 71, 3, 3)
(504, 71, 6, 3)
(460, 71, 3, 4)
(455, 71, 3, 4)
(426, 71, 3, 3)
(407, 71, 15, 11)
(351, 71, 5, 3)
(314, 71, 23, 8)
(308, 71, 5, 4)
(294, 71, 5, 3)
(288, 71, 5, 3)
(263, 71, 23, 6)
(224, 71, 5, 4)
(181, 71, 6, 5)
(167, 71, 8, 7)
(160, 71, 3, 3)
(2445, 70, 4, 4)
(2398, 70, 3, 3)
(2383, 70, 4, 3)
(2262, 70, 3, 3)
(2230, 70, 3, 3)
(2222, 70, 4, 3)
(2216, 70, 5, 3)
(2214, 70, 3, 3)
(1998, 70, 3, 3)
(1991, 70, 3, 3)
(1978, 70, 7, 4)
(1975, 70, 4, 3)
(1951, 70, 7, 3)
(1946, 70, 4, 3)
(1942, 70, 3, 3)
(1932, 70, 7, 3)
(1892, 70, 5, 3)
(1841, 70, 3, 3)
(1809, 70, 5, 3)
(1580, 70, 3, 3)
(1530, 70, 5, 3)
(1501, 70, 4, 3)
(1494, 70, 4, 3)
(1487, 70, 6, 4)
(1454, 70, 3, 3)
(1449, 70, 4, 3)
(1375, 70, 6, 3)
(1357, 70, 7, 4)
(1334, 70, 3, 3)
(1295, 70, 4, 4)
(1279, 70, 10, 4)
(1185, 70, 4, 3)

KeyboardInterrupt: 